<a href="https://colab.research.google.com/github/centeno/covid-impact-scrapper/blob/master/step01_collection_data_GoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalação das dependencias
#### SEMPRE RODAR ANTES

In [1]:

#!apt-get update # to update ubuntu to correctly run apt install

# scraper
!pip install selenium
!apt install chromium-chromedriver
!pip install webdriver-manager
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium msticpy tld folium
!pip pymongo pymongo[srv] dnspython

# geocoder ipaddress
!pip install msticpy

# get domain
!pip3 install tld


# spacy
!pip install -U pip setuptools wheel
!pip install -U spacy
#!python -m spacy download en
#!python -m spacy download pt
!python -m spacy download en_core_web_sm
# spacy contextual Spell Check
!pip install contextualSpellCheck
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3




     |████████████████████████████████| 911kB 6.3MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 86.0 MB of archives.
After this operation, 298 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 91.0.4472.101-0ubuntu0.18.04.1 [1,124 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 91.0.4472.101-0ubuntu0.18.04.1 [76.1 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe a

ERROR: unknown command "pymongo"
     |████████████████████████████████| 102kB 4.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.12.1 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 1.6MB 5.1MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 70 kB/s 
     |████████████████████████████████| 9.1 MB 39.1 MB/s 
     |████████████████████████████████| 618 kB 35.1 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 456 kB 42.5 MB/s 
     |████████████████████████████████| 113 k

### Import das dependencias

In [2]:
import os
import glob
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, date, timedelta

import urllib
import socket
from msticpy.sectools.geoip import IPStackLookup

import spacy
import contextualSpellCheck

import pymongo
import pandas as pd
from datetime import datetime as dt
from datetime import date, timedelta
from pandas.tseries.offsets import DateOffset
import dns

import tld
from tld import get_tld, get_fld

from google.colab import drive

### Connect google drive

In [3]:
# Codigo para montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Caminho no google drive
path = "/content/drive/My Drive/covid19impact/data/2020fake/01_collect/en/"

# Listar os arquivos da pasta no google drive
os.chdir(path)
!ls

2019-09-30.csv	2019-10-01.csv	2019-10-02.csv	2019-10-03.csv	2019-10-04.csv


### Import chaves de api

In [5]:
# drive
keys = pd.read_csv("/content/drive/My Drive/covid19impact/keys/credentials.csv", sep=';')
access_token_ipstack = keys.loc[0,"key"]
access_token_pymongo = keys.loc[1,"key"]

### Create functions

In [6]:
import pandas as pd
GSE_RESULTS_CLASS = 'ZINbbc xpd O9g5cc uUPGi'
GSE_MAX_RESULTS = 200

NLP = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(NLP)

def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36')    
  return webdriver.Chrome('chromedriver',options=options)

def page_source(query, date):
  d = date.strftime('%Y%m%d')
  q = "{} after:{} and before:{}".format(query, date.strftime('%Y-%m-%d'), date.strftime('%Y-%m-%d'))
  url = "https://www.google.com/search?q={}&sort=date:r:{}:{}&num={}".format(q, d, d, GSE_MAX_RESULTS)
  driver = get_driver()
  driver.get(url)
  soup = BeautifulSoup(driver.page_source, 'html.parser') 
  return driver.page_source
  
def fetch_result(date, item):
  try:
    return {
        'date': date.strftime('%Y-%m-%d'),
        'header' : item.select_one('h3').text,
        'summary': item.findChildren('div', recursive=False)[-1].text,
        'link' : item.find('a', href=True)['href'].replace('/url?q=', ''),
    }
  except:
    return None

def search(query, date):
  soup = BeautifulSoup(page_source(query, date), 'html.parser') 
  html_results = soup.find_all('div', {'class': GSE_RESULTS_CLASS })
  results = [fetch_result(date, r) for r in html_results]
  return pd.DataFrame(list(filter(None, results)))

def fetch_domain(df):
  df['protocol'],df['domain'],df['path'],df['query'],df['fragment'] = zip(*[urllib.parse.urlsplit(x) for x in df.link])
  return df

def fetch_hostname(df): 
  urls = df['link']
  hostnames = []
  for url in urls:
    try:
      res = get_tld(url, as_object=True)
      hostnames.append(res.fld)
    except:
      res = np.nan
      hostnames.append(res)
  df['host_name'] = hostnames
  return df

def fetch_ipaddress(df):
  domains = df['domain']
  ipaddress = []
  for i in domains:
      i_strip = i.strip()
      try:
          ip = socket.gethostbyname(i_strip)
          ipaddress.append(ip)
      except socket.gaierror:
        ipaddress.append('127.0.0.1')
  df['ip_address'] = ipaddress
  return df

def fetch_location(df):
  iplocation = IPStackLookup(api_key=access_token_ipstack)
  df = iplocation.df_lookup_ip(df, column="ip_address")
  return df

def fetch_nlp(df):
  df['header_context_true'] = df['header'].apply(lambda x: str(NLP(x)))
  df['summary_context_true'] = df['summary'].apply(lambda x: str(NLP(x)))
  return df

def save(df):
  datamax = str(df['date'].max())
  fileName = path + datamax + '.csv'
  df.to_csv(fileName, sep=';', encoding='UTF-8')
  
def next_data(df):
  df['date'] = pd.to_datetime(df['date'], dayfirst = False)
  lst_date = df['date'].max()
  nxt_date = (lst_date.date() + DateOffset(days = 1)).date()
  return nxt_date

### APENAS PARA MONGODB ###
#def save(df):
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = df.astype('str')
#  db.impacts.insert_many(df.to_dict('records'))

#def load():
#  client = pymongo.MongoClient(access_token_pymongo)
#  db = client.covid_impact
#  df = pd.DataFrame.from_records(db.impacts.find())
# return df

### Create strings

In [9]:
keyword = '"covid-19" OR "covid" OR "coronavirus" AND “impact" OR "disaster" OR "resilience" AND “software” OR "software engineer" OR "agile" OR "information industry" OR "IT" OR "disaster informatics" OR "crisis informatics" OR "productivity" OR "remote" OR "work" OR "future of work"'

### Merge csv's or SCRAPER new search

In [8]:
# merge multiples csv's files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
files = len(all_filenames)

if files >= 1:
  #combine all files in path
  df = pd.concat([pd.read_csv(f, sep=';', parse_dates=["date"], engine="python", encoding='UTF-8') for f in all_filenames ])
else:
  #get first date
  first_date = date(2019, 9, 30)
  #get seach
  df = search(keyword,first_date)
  #get domain
  df = fetch_domain(df)
  #get hostname
  df = fetch_hostname(df)
  #get ipaddress
  df= fetch_ipaddress(df)
  #get geocoded ipaddress
  df = fetch_location(df)
  #save
  save(df)  

#print df
df.tail()

,Unnamed: 0,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
99,99,2019-10-04,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ...",https://www.esri.com/en-us/disaster-response/o...,https,www.esri.com,/en-us/disaster-response/overview&sa=U&ved=2ah...,NaN,NaN,esri.com,23.48.47.116,US,United States,Virginia,Ashburn,-77.474197,39.043701,16625.0,set(),geolocation,{},23.48.47.116,"Esri Disaster Response Program | Maps, Softwar...","When a disaster or crisis strikes, we're here ..."
100,100,2019-10-04,The impact of work loss on mental and physical...,"Sep 9, 2020 · Participants Australians aged 18...",https://www.medrxiv.org/content/10.1101/2020.0...,https,www.medrxiv.org,/content/10.1101/2020.09.06.20189514v1&sa=U&ve...,NaN,NaN,medrxiv.org,104.18.16.209,US,United States,California,San Jose,-121.838226,37.330528,13335.0,set(),geolocation,{},104.18.16.209,The impact of work loss on mental and physical...,"Sep 9, 2020 · Participants Australians aged 18..."
101,101,2019-10-04,Public Assistance Disaster-Specific Guidance -...,This page contains disaster-specific guidance ...,https://www.fema.gov/media-collection/public-a...,https,www.fema.gov,/media-collection/public-assistance-disaster-s...,NaN,NaN,fema.gov,104.117.14.155,US,United States,Virginia,Ashburn,-77.474197,39.043701,16625.0,set(),geolocation,{},104.117.14.155,Public Assistance Disaster-Specific Guidance -...,This page contains disaster-specific guidance ...
102,102,2019-10-04,COVID-19 impact: Work from home more appealing...,COVID-19 impact: Work from home more appealing...,https://www.usatoday.com/story/tech/2021/03/25...,https,www.usatoday.com,/story/tech/2021/03/25/covid-remote-work-offic...,NaN,NaN,usatoday.com,151.101.2.62,US,United States,California,San Francisco,-122.392860,37.767849,54113.0,set(),geolocation,{},151.101.2.62,COVID-19 impact: Work from home more appealing...,COVID-19 impact: Work from home more appealing...
103,103,2019-10-04,"COVID-19 Pandemic, Geospatial Information, and...",Land Administration and Authoritative Geospati...,https://www.routledge.com/COVID-19-Pandemic-Ge...,https,www.routledge.com,/COVID-19-Pandemic-Geospatial-Information-and-...,NaN,NaN,routledge.com,104.16.26.63,US,United States,Virginia,Ashburn,-77.474197,39.043701,13335.0,set(),geolocation,{},104.16.26.63,"COVID-19 Pandemic, Geospatial Information, and...",Land Administration and Authoritative Geospati...


### SCRAPER new search

In [10]:
#get next data
next_date = next_data(df)

#get seach
df_new = search(keyword, next_date)

#get domain
df_new = fetch_domain(df_new)

#get hostname
df_new = fetch_hostname(df_new)

#get ipaddress
df_new = fetch_ipaddress(df_new)

#get geocoded ipaddress
df_new = fetch_location(df_new)

#get nlp contextualSpellCheck 
df_new = fetch_nlp(df_new)

#save df new
save(df_new)

#print new df
df_new.head(2)



,date,header,summary,link,protocol,domain,path,query,fragment,host_name,ip_address,CountryCode,CountryName,State,City,Longitude,Latitude,Asn,edges,Type,AdditionalData,IpAddress,header_context_true,summary_context_true
0,2019-10-05,How the coronavirus pandemic will impact the f...,"Apr 29, 2020 · For Americans, the economic imp...",https://www.cnbc.com/2020/04/29/how-the-corona...,https,www.cnbc.com,/2020/04/29/how-the-coronavirus-pandemic-will-...,,,cnbc.com,23.46.244.122,US,United States,Georgia,Lithia Springs,-84.645157,33.759171,16625.0,{},geolocation,{},23.46.244.122,How the coronavirus pandemic will impact the f...,"Apr 29, 2020 · For Americans, the economic imp..."
1,2019-10-05,Coronavirus' business impact: Evolving perspec...,4 days ago · There's a lot to look forward to ...,https://www.mckinsey.com/business-functions/ri...,https,www.mckinsey.com,/business-functions/risk/our-insights/covid-19...,,,mckinsey.com,23.220.87.194,US,United States,Georgia,Atlanta,-84.388283,33.798458,16625.0,{},geolocation,{},23.220.87.194,Coronavirus' business impact: Evolving perspec...,4 days ago · There's a lot to look forward to ...
